In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

# os.environ["GOOGLE_API_KEY"] = getpass.getpass()

import dotenv

dotenv.load_dotenv()

True

In [12]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
llm = GoogleGenerativeAI(model='gemini-pro')

In [10]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))


In [14]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("what are types of memory")

'There are three main types of memory: sensory memory, short-term memory, and long-term memory. Sensory memory stores information for a very short period of time, while short-term memory stores information for a few seconds or minutes. Long-term memory stores information for a long period of time, and can be divided into two types: explicit memory and implicit memory.'

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f39099d47e0> >

In [23]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()